### Basic library imports

In [1]:
import os
import pandas as pd
import re
import requests
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

### Read Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('/content/drive/My Drive/dataset/train.csv')
test = pd.read_csv('/content/drive/My Drive/dataset/test.csv')
sample_test = pd.read_csv('/content/drive/My Drive/dataset/sample_test.csv')
sample_test_out = pd.read_csv('/content/drive/My Drive/dataset/sample_test_out.csv')
output = pd.read_csv('/content/drive/My Drive/dataset/output_sanity.csv')

Mounted at /content/drive


<ipython-input-2-9e9372db087f>:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  output = pd.read_csv('/content/drive/My Drive/dataset/output_sanity.csv')


In [3]:
unique_values_count = test['group_id'].nunique()
unique_value = test['group_id'].unique()

print(f'Number of unique values: {unique_values_count}')
unique_value

Number of unique values: 924


array([156839, 792578, 478357, 953313, 276611, 648011, 279307, 569206,
       348551, 442321, 751532, 498074, 178958, 752266, 965518, 957050,
       486636, 329793, 113134, 601746, 244283, 249345, 955292, 916768,
       297918, 709627, 997176, 558806, 801829, 630390, 488883, 933453,
       411423, 469317, 860821, 311997, 932012, 120219, 961822, 318770,
       524117, 628971, 219211, 267482, 849273, 308856, 982714, 365637,
       494500, 931856, 150913, 594224, 658003, 913156, 452717, 449805,
       704724, 825954, 386873, 695925, 640408, 521308, 312608, 507988,
       519155, 308671, 861555, 658892, 594191, 900671, 783898, 276075,
       801837, 926134, 149708, 939129, 966823, 825239, 635528, 625310,
       462731, 243137, 823083, 254449, 171418, 781426, 433914, 292475,
       893692, 120569, 369753, 483370, 152057, 983323, 302672, 810266,
       375816, 436746, 639090, 858439, 276700, 952353, 922709, 178778,
       501250, 963595, 387046, 347404, 764417, 745326, 245959, 525429,
      

## **Data Preprocessing**

## **Text Extraction using Tesseract OCR**

In [4]:
entity_unit_map = {
    'width': {'centimetre', 'centimeter', 'cm', 'foot', 'ft', 'inch', 'in', 'metre', 'meter', 'm', 'millimetre', 'millimeter', 'mm', 'yard', 'yd'},
    'depth': {'centimetre', 'centimeter', 'cm', 'foot', 'ft', 'inch', 'in', 'metre', 'meter', 'm', 'millimetre', 'millimeter', 'mm', 'yard', 'yd'},
    'height': {'centimetre', 'centimeter', 'cm', 'foot', 'ft', 'inch', 'in', 'metre', 'meter', 'm', 'millimetre', 'millimeter', 'mm', 'yard', 'yd'},
    'item_weight': {'gram', 'g', 'kilogram', 'kg', 'microgram', 'µg', 'milligram', 'mg', 'ounce', 'oz', 'pound', 'lb', 'ton'},
    'maximum_weight_recommendation': {'gram', 'g', 'kilogram', 'kg', 'microgram', 'µg', 'milligram', 'mg', 'ounce', 'oz', 'pound', 'lb', 'ton'},
    'voltage': {'kilovolt', 'kv', 'millivolt', 'mv', 'volt', 'v'},
    'wattage': {'kilowatt', 'kw', 'watt', 'w'},
    'item_volume': {
        'centilitre', 'cl', 'cubic foot', 'ft³', 'cubic inch', 'in³', 'cup', 'decilitre', 'dl',
        'fluid ounce', 'fl oz', 'gallon', 'gal', 'imperial gallon', 'litre', 'liter', 'l', 'microlitre', 'µl',
        'millilitre', 'ml', 'pint', 'pt', 'quart', 'qt'
    }
}


allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [5]:
unit_full_form_map = {
    'cm': 'centimeter', 'centimetre': 'centimeter',
    'mm': 'millimeter', 'm': 'meter', 'ft': 'foot', 'in': 'inch', 'yd': 'yard',
    'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'µg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'ton': 'ton',
    'kv': 'kilovolt', 'mv': 'millivolt', 'v': 'volt',
    'w': 'watt', 'kw': 'kilowatt',
    'cl': 'centilitre', 'ft³': 'cubic foot', 'in³': 'cubic inch', 'l': 'liter', 'ml': 'millilitre', 'µl': 'microlitre',
    'pt': 'pint', 'qt': 'quart', 'fl oz': 'fluid ounce', 'gal': 'gallon'
}

allowed_units = set(unit_full_form_map.keys())

In [6]:
# Import necessary libraries
# import pytesseract
from PIL import Image
import cv2

# Install Pillow and OpenCV (if not already installed)
!pip install pillow
!pip install opencv-python
!pip install paddlepaddle paddleocr
# !pip install easyocr



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=88ded62aca4718f8b4e37c493e7f778c99547bcd53d5a87ba74b4ceab6556dff
  Stored in d

## **Entity Value Extraction using Regular Expressions**

Paddle OCR

In [7]:
!pip install pytesseract


In [8]:
df = pd.DataFrame(test)
# Create a new column to identify changes in group_id
df['group_change'] = (df['group_id'] != df['group_id'].shift()).astype(int)
df_filtered = df[df['group_change'] != 0]
df['prediction'] = output['prediction']
print(len(df_filtered))

89879


In [9]:
import re
from collections import Counter


# # Function to convert units to full form and lowercase
# def convert_unit_to_full_form(unit):
#     unit = unit.lower()  # Convert to lowercase
#     return unit_full_form_map.get(unit, unit)  # Convert to full form if available

# Helper function to parse values and convert units to their full form
def extract_values(ocr_text):
    # Regex to find values with units like 45.8 cm, 0.8 inch
    pattern = re.compile(r"(\d*\.?\d+)\s*(cm|inch|centimetre|centimeter|cm|foot|ft|inch|in|metre|meter|m|millimetre|millimeter|mm|yard|yd|km|kilometer|kilometre)")
    matches = pattern.findall(ocr_text)
    # Convert the unit to full form in lowercase
    return [(float(value), convert_unit_to_full_form(unit)) for value, unit in matches]

# Main function to determine height, width, and depth
def get_dimensions(ocr_text):
    extracted_values = extract_values(ocr_text)

    if not extracted_values:
        return "", "", ""

    # Count the occurrence of each unit and pick the most frequent one
    units = [unit for _, unit in extracted_values]
    most_common_unit = Counter(units).most_common(1)[0][0]

    # Filter values that match the most frequent unit
    values_in_common_unit = [value for value, unit in extracted_values if unit == most_common_unit]

    # If less than 2 values in the most common unit, set both width and depth to None
    if len(values_in_common_unit) < 2:
            height = f"{values_in_common_unit[0]} {most_common_unit}" if values_in_common_unit else ""
            width = ""
            depth = ""

            return height, width, depth

    # If less than 3 values in the most common unit, set depth to None
    if len(values_in_common_unit) < 3:
            values_in_common_unit.sort(reverse=True)
            height = f"{values_in_common_unit[0]} {most_common_unit}"
            width = f"{values_in_common_unit[1]} {most_common_unit}" if len(values_in_common_unit) > 1 else ""
            depth = ""
            return height, width, depth

    # Sort the values in descending order and assign to height, width, and depth
    values_in_common_unit.sort(reverse=True)
    height = f"{values_in_common_unit[0]} {most_common_unit}"
    width = f"{values_in_common_unit[1]} {most_common_unit}" if len(values_in_common_unit) > 1 else ""
    depth = f"{values_in_common_unit[2]} {most_common_unit}" if len(values_in_common_unit) > 2 else ""

    return height, width, depth

# Example Usage
# ocr_text = " .8inch, 87 inch, 3m, 36m, 76 km , 23Km, 23kilometre "
# print(dimensions)

In [10]:
import pandas as pd
import re
from paddleocr import PaddleOCR
import requests
from PIL import Image, ImageEnhance
from io import BytesIO
import numpy as np
import pytesseract  # Import pytesseract for Tesseract OCR

# Initialize the PaddleOCR model (English and Chinese support)
ocr_model = PaddleOCR(use_angle_cls=True, lang='en')

# Function to handle common mistakes in units
def common_mistake(unit):
    if unit is None:  # Check if unit is None
        return None
    if unit in allowed_units:
        return unit
    if unit.replace('ter', 'tre') in allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def add_space_between_alpha_and_number(text):
    # Regular expression pattern to find a letter followed by a number
    pattern = r"([a-zA-Z])(\d)"
    # Replace matches with the letter followed by a space and then the number
    spaced_text = re.sub(pattern, r"\1 \2", text)
    return spaced_text

# Function to extract numeric values from OCR-extracted text
# def extract_numeric_values(text):
#     return re.findall(r"\d*\.\d+|\d+", text)

def extract_numeric_values_and_units(text, entity_units):
    # Modify the regex to look for numbers followed by units (with or without space)
    pattern = r"(\d*\.\d+|\d+)\s*({})".format('|'.join(map(re.escape, entity_units)))
    matches = re.findall(pattern, text)

    cleaned_matches = []
    for num, unit in matches:
        # Check if there's a dot before the number without any leading digits (e.g., ".90lb")
        if num.startswith("."):
            # Remove the dot if no number precedes it (to handle cases like ".90lb")
            cleaned_num = num[1:] if len(num) > 1 else num
        else:
            cleaned_num = num

        # Append cleaned number and unit
        cleaned_matches.append((float(cleaned_num), unit))

    return cleaned_matches

# Function to extract OCR text from an image URL using PaddleOCR or Tesseract as a fallback
def extract_ocr_text_from_image(image_link, use_tesseract=False):
    try:
        # Download the image
        response = requests.get(image_link)
        img = Image.open(BytesIO(response.content))
        img = img.convert("RGB")  # Ensure the image is in RGB mode

        if use_tesseract:
            # Use Tesseract to extract text if specified
            ocr_text = pytesseract.image_to_string(img)
        else:
            # Use PaddleOCR to get text
            ocr_results = ocr_model.ocr(np.array(img))
            # Extract the detected text and join all results
            ocr_text = ' '.join([item[1][0] for line in ocr_results for item in line])

        # Check if OCR text is empty or contains only whitespace
        if not ocr_text.strip():
            print(f"No text detected in image: {image_link}")
            return ""  # Return empty string if no text is detected

        return ocr_text
    except Exception as e:
        print(f"Error extracting OCR text: {e}")
        return ""  # Return empty string if an error occurs

# Function to convert unit abbreviations to full forms
# def convert_unit_to_full_form(unit):
#     return unit_full_form_map.get(unit, unit)  # If the unit is not in the map, return it as is
# Function to convert units to full form and lowercase
def convert_unit_to_full_form(unit):
    unit = unit.lower()  # Convert to lowercase
    return unit_full_form_map.get(unit, unit)  # Convert to full form if available

def extract_numeric_values_and_units(text, entity_units):
    # Modify the regex to look for numbers followed by units (with or without space)
    pattern = r"(\d*\.\d+|\d+)\s*({})".format('|'.join(map(re.escape, entity_units)))
    matches = re.findall(pattern, text)

    cleaned_matches = []
    for num, unit in matches:
        # Check if there's a dot before the number without any leading digits (e.g., ".90lb")
        if num.startswith("."):
            # Remove the dot if no number precedes it (to handle cases like ".90lb")
            cleaned_num = num[1:] if len(num) > 1 else num
        else:
            cleaned_num = num

        # Append cleaned number and unit
        cleaned_matches.append((float(cleaned_num), unit))

    return cleaned_matches

# Function to match numeric values to the entity based on extracted values and units
def match_value_to_entity(numeric_values_with_units, entity_name):
    # Get the valid units for the given entity name
    valid_units = entity_unit_map.get(entity_name.lower(), [])  # Use lowercase for entity matching

    # Iterate over extracted values and units
    for value, unit in numeric_values_with_units:
        if unit in valid_units:
            return value, unit  # Return the first valid match

    return None, None  # If no match is found

# Updated process_dataset to use the new matching logic
def process_dataset(df, output):
    # Initialize count of matches
    image_no = 1
    ocr_text = ''

    for index, row in df.iterrows():
        print("AMAZON")
        image_link = row['image_link']
        entity_name = row['entity_name']
        result = row['prediction']

        if result == '' or (entity_name == 'height' or entity_name == 'width' or entity_name == 'depth'):

            print(f"\nIndex Image: {image_no}\n")
            image_no += 1

            if row['group_change'] == 0:
              print(f"Skipping OCR extraction for index {index} as group_id is repeated.")

            elif row['group_change'] == 1:

            # Extract OCR text from the image using Tesseract first
                ocr_text = extract_ocr_text_from_image(image_link, use_tesseract=True)

                if not ocr_text:  # If OCR text is still empty after both OCR attempts
                    print(f"Tesseract failed, trying PaddleOCR for Image: {image_link}")
                    ocr_text = extract_ocr_text_from_image(image_link)


            if ocr_text:
                # Add a space between alphabetic characters and numbers
                ocr_text = add_space_between_alpha_and_number(ocr_text)

                # Extract numeric values from OCR text
                ocr_text = ocr_text.lower()
                print(f"OCR Text: {ocr_text}")

                if(entity_name == 'height' or entity_name == 'width' or entity_name == 'depth') :
                    height, width, depth = get_dimensions(ocr_text)
                    entity_val = ""

                    if(entity_name == 'height') :
                      entity_val = height

                    elif (entity_name == 'width') :
                      entity_val = width

                    else :
                      entity_val = depth

                    output.loc[index, 'prediction'] = entity_val
                    # output({"index": index, "entity_name": entity_name, "entity_value": entity_val})


                else:
                  entity_units = entity_unit_map.get(entity_name, allowed_units)

                  # Extract numeric values and their corresponding units
                  numeric_values_with_units = extract_numeric_values_and_units(ocr_text, entity_units)

                  # Match the numeric values to the entity
                  value, unit = match_value_to_entity(numeric_values_with_units, entity_name)
                  unit = common_mistake(unit)

                  if value and unit:
                      # Convert the unit to its full form before appending
                      full_unit = convert_unit_to_full_form(unit)
                      # cnt += 1  # Increment count of successful matches
                      output.loc[index, 'prediction'] = f"{value} {full_unit}"
                      # results.append({"index": index, "entity_name": entity_name, "entity_value": f"{value} {full_unit}"})
                  else:
                      # If no match is found, keep the entity_value empty
                      output.loc[index, 'prediction'] = ""
                      # results.append({"index": index, "entity_name": entity_name, "entity_value": ""})
                      print(f"No match found for Entity: {entity_name} in Image: {image_link}")

            else:
                # If OCR text extraction fails, keep entity_value empty
                output.loc[index, 'prediction'] = ""
                # results.append({"index": index, "entity_name": entity_name, "entity_value": ""})
                print(f"Failed to extract OCR text for Image: {image_link}")

    return output


# Sample DataFrame (replace with actual data)
start_index = 0
stop_index = 131188

# Create a new column to identify changes in group_id
# # Calculate the number of unique group_id sequences

df_selected = df[start_index:stop_index].copy()
df_selected['group_change'] = (df_selected['group_id'] != df_selected['group_id'].shift()).astype(int)

for i in range(start_index, stop_index):
    df.at[i, 'group_change'] = df_selected.at[i, 'group_change']

# Process the dataset
process_dataset(df[start_index:stop_index], output[start_index:stop_index])

# Display the final output
output[start_index:stop_index]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:16<00:00, 237kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:18<00:00, 553kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 139kiB/s]

[2024/09/16 02:13:38] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

Streaming output truncated to the last 5000 lines.

Skipping OCR extraction for index 11316 as group_id is repeated.
OCR Text: 10.5cm 300ml 4.13inch 0 7,2cm 0 2.83inch
AMAZON

Index Image: 6539

Error extracting OCR text: tesseract is not installed or it's not in your PATH. See README file for more information.
Tesseract failed, trying PaddleOCR for Image: https://m.media-amazon.com/images/I/41Y1qmg6zhL.jpg
[2024/09/16 03:09:02] ppocr DEBUG: dt_boxes num : 6, elapsed : 0.30008387565612793
[2024/09/16 03:09:02] ppocr DEBUG: cls num  : 6, elapsed : 0.04656028747558594
[2024/09/16 03:09:03] ppocr DEBUG: rec_res num  : 6, elapsed : 0.3599233627319336
OCR Text: 7 inch .57 10.07 14.17 inch
AMAZON

Index Image: 6540

Error extracting OCR text: tesseract is not installed or it's not in your PATH. See README file for more information.
Tesseract failed, trying PaddleOCR for Image: https://m.media-amazon.com/images/I/41Y2vaD+hqL.jpg
[2024/09/16 03:09:03] ppocr DEBUG: dt_boxes num : 2, elapsed : 0

KeyboardInterrupt: 

In [11]:
len(output)

131187

In [13]:
outputs = output[start_index: 12230]
outputs.to_csv('output_final.csv', index=False)

Easy OCR